# Analysis of nursing home deficiencies

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
raw_df = pd.read_csv(
    "../data/health_deficiencies.csv",
    parse_dates = ["survey_date", "correction_date", "processing_date"],
    dtype = {
        "federal_provider_number": "object",
        "provider_zip_code": "object",
        "deficiency_tag_number": "object",
        "inspection_cycle": "object"
    }
)

raw_df.head(1)

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,survey_date,survey_type,deficiency_prefix,deficiency_category,...,deficiency_corrected,correction_date,inspection_cycle,standard_deficiency,complaint_deficiency,location,processing_date,:@computed_region_f3tr_pr43,:@computed_region_nwen_78xc,:@computed_region_csmy_5jwy
0,145121,ALTON MEMORIAL REHAB & THERAPY,1251 COLLEGE AVENUE,ALTON,IL,62002,2018-09-17,Health,F,Resident Rights Deficiencies,...,"Deficient, Provider has date of correction",2018-10-03,2,Y,False,"{'latitude': '38.904106', 'longitude': '-90.16...",2020-03-01,1749.0,1749.0,34.0


In [3]:
#  Mark the deficiencies we care about
df = (
    raw_df
    .assign(
        has_infection = lambda x: x["deficiency_tag_number"].isin(["0880", "0441"]),
        has_ulcer = lambda x: x["deficiency_tag_number"].isin(["0686", "0314"]),
        has_med = lambda x: x["deficiency_tag_number"].isin(["0760", "0333"])
    )
)

How many Nursing Homes are there?

In [4]:
total = df['federal_provider_number'].nunique()

total 

15302

Get survey dataset so we can calculate the universe of facilities within a given inspection period

In [5]:
surveys = (
    pd
    .read_csv(
        "../data/survey.csv",
        dtype = {"federal_provider_number": "object"},
        parse_dates = ["health_survey_date"]
    )
)

What is the date range of one cycle?

In [6]:
(
    surveys
    .loc[ lambda x: x["inspection_cycle"] == 1 ]
    ['health_survey_date']
    .describe()
)

count                   15446
unique                    542
top       2019-11-21 00:00:00
freq                      161
first     2017-05-18 00:00:00
last      2020-02-25 00:00:00
Name: health_survey_date, dtype: object

Two cycles?

In [7]:
(
    surveys
    .loc[ lambda x: x["inspection_cycle"].isin([1,2]) ]
    ['health_survey_date']
    .describe()
)

count                   30799
unique                    966
top       2018-11-29 00:00:00
freq                      169
first     2016-02-02 00:00:00
last      2020-02-25 00:00:00
Name: health_survey_date, dtype: object

In an email, CMS said they surveyed 96 facilities between March 23 and April 10 (19 days, including the end date). How many facilities did CMS cover last year in this time period?

In [8]:
# Time difference between CMS dates
datetime(2020, 4, 10) - datetime(2020, 3, 23)

datetime.timedelta(days=18)

In [9]:
# Find a similar timeframe last year (starting on a Monday)
datetime(2019, 3, 25) + timedelta(days = 18)

datetime.datetime(2019, 4, 12, 0, 0)

In [10]:
# Calculate same time period last year, including the last day to make 19 days
last19 = (
    surveys
    .loc[
        lambda x: (x["health_survey_date"] >= "2019-03-25") & (x["health_survey_date"] <= "2019-04-12")
    ]
)

len(last19)

925

Aggregate violations

In [11]:
(
    last19
    [[
        "total_number_of_health_deficiencies",
        "count_of_immediate_jeopardy_deficiencies_on_health_survey",
    ]]
    .sum()
    .to_frame("Deficiencies")
)

,Deficiencies
total_number_of_health_deficiencies,6388
count_of_immediate_jeopardy_deficiencies_on_health_survey,1127


## Examining "immediate jeopardy" violations

How many "immmediate jeopardy" (I, J, K) violations happen?

In [12]:
# Make a df with the flags we need
ij = (
    raw_df
    .assign(
        year = lambda x: x["survey_date"].dt.year,
        immediate_jeopardy = lambda x: x["scope_severity_code"].isin(["J", "K", "L"]),
        isolated = lambda x: x["scope_severity_code"] == "J",
        pattern = lambda x: x["scope_severity_code"] == "K",
        widespread = lambda x: x["scope_severity_code"] == "L"   
    )
)

Violations since 2017

In [13]:
(
    ij
    .loc[
        lambda x: x["survey_date"] >= "2017-01-01"
    ]
    ["immediate_jeopardy"]
    .sum()
)

5594

Violations just in 2019

In [14]:
(
    ij
    .loc[
        lambda x: x["survey_date"] >= "2019-01-01"
    ]
    ["immediate_jeopardy"]
    .sum()
)

1869

Number of homes with immediate jeopardy violations since 2017

In [15]:
(
    ij
    .loc[
        lambda x: x["survey_date"] >= "2017-01-1"
    ]
    .loc[
        lambda x: x["immediate_jeopardy"] == True
    ]
    ["federal_provider_number"]
    .nunique()
)

2485

## Examining specific deficiencies

Raw numbers on facilities with ulcers since 2018

In [16]:
(
    df
    .loc[ lambda x: x["standard_deficiency"] == "Y" ]
    .loc[ lambda x: x["survey_date"] >= "2018-01-01" ]
    .loc[ lambda x: x["has_ulcer"]]
    ["federal_provider_number"]
    .nunique()
)

3376

Raw numbers on facilities with medicine errors since 2018

In [17]:
(
    df
    .loc[ lambda x: x["standard_deficiency"] == "Y" ]
    .loc[ lambda x: x["survey_date"] >= "2018-01-01" ]
    .loc[ lambda x: x["has_med"] ]
    ["federal_provider_number"]
    .nunique()
)

1250

Count health inspection deficiencies per cycle

In [18]:
# helper function to return count and percent for deficiencies given a filtered dataframe
def count_cycles(df):
    
    cycles = df['inspection_cycle'].unique()
    
    total = surveys.loc[
        lambda x: x["inspection_cycle"].isin(cycles)
    ]['federal_provider_number'].nunique()

    print("total =", total)
    
    return (
        df
        .groupby("federal_provider_number")
        .pipe(lambda grp: pd.DataFrame(
            {
                "has infection deficiencies": grp["has_infection"].any(),
                "has pressure sore deficiencies": grp["has_ulcer"].any(),
                "has significant medicine errors": grp["has_med"].any()
            })
        )
        .sum()
        .to_frame("count")
        .assign(
            percent = lambda frame: (frame["count"]/total) * 100 
        )
    )
    

Last standard inspection

In [19]:
count_cycles(
   df
    .loc[
        lambda x: x["standard_deficiency"] == "Y"
    ]
    .loc[
        lambda x: x["inspection_cycle"] == "1"
    ]
)

total = 15446


,count,percent
has infection deficiencies,6307,40.832578
has pressure sore deficiencies,1932,12.508093
has significant medicine errors,712,4.609608


Last two standard inspections

In [20]:
count_cycles(
    df
    .loc[
        lambda x: x["inspection_cycle"].isin(["1","2"])
    ]
    .loc[
        lambda x: x["standard_deficiency"] == "Y"
    ]
)

total = 15446


,count,percent
has infection deficiencies,9421,60.993137
has pressure sore deficiencies,3549,22.976822
has significant medicine errors,1327,8.591221


Last inspection cycle, complaints included

In [21]:
count_cycles(
    df
    .loc[
        lambda x: x["inspection_cycle"].isin(["1"])
    ]
)

total = 15446


,count,percent
has infection deficiencies,6598,42.716561
has pressure sore deficiencies,2440,15.796970
has significant medicine errors,1102,7.134533


Last two inspection cycles, complaints included

In [22]:
count_cycles(
    df
    .loc[
        lambda x: x["inspection_cycle"].isin(["1","2"])
    ]
)

total = 15446


,count,percent
has infection deficiencies,9763,63.207303
has pressure sore deficiencies,4297,27.819500
has significant medicine errors,1994,12.909491


Count infection control OR ulcer violation

In [23]:
total = surveys.loc[
    lambda x: x["inspection_cycle"].isin(["1", "2"])
]['federal_provider_number'].nunique()

print("total =", total)

(
    df
    .loc[
        lambda x: x["inspection_cycle"].isin(["1","2"])
    ]
    .groupby("federal_provider_number")
    .pipe(lambda grp: pd.DataFrame({
            "has infection control or ulcer": grp["has_infection"].any() | grp["has_ulcer"].any()
        })
    )
    .sum()
    .to_frame("count")
    .assign(
        percent = lambda frame: (frame["count"]/total) * 100 
    )
)

total = 

15446


,count,percent
has infection control or ulcer,10773,69.746213


Proportion of facilities with standard deficiencies by state since last inspection

In [24]:
state_lookup = (
    surveys
    .loc[
        lambda x: x["inspection_cycle"].isin([1,2])
    ]
    .groupby('provider_state')
    .agg(
        {"federal_provider_number": "nunique"}
    )
    .reset_index()
    .rename(
        columns = {"federal_provider_number": "total_facilities"}
    )
)

state_lookup.head()

,provider_state,total_facilities
0,AK,19
1,AL,228
2,AR,225
3,AZ,146
4,CA,1193


In [25]:
(
    df
    .loc[
        lambda x: x["inspection_cycle"].isin(["1", "2"])
    ]
    .loc[
        lambda x: x["standard_deficiency"] == "Y"
    ]
    .groupby(["provider_state", "federal_provider_number"])
    # get a bool for any provider with thse deficiencies
    .pipe(lambda grp: pd.DataFrame(
        {
            "infection deficiencies": grp["has_infection"].any(),
            "sore deficiencies": grp["has_ulcer"].any(),
            "med deficiencies": grp["has_med"].any()
        })
    )
    .reset_index()
    .groupby("provider_state")
    # count the bools
    .agg({
        "infection deficiencies": "sum",
        "sore deficiencies": "sum",
        "med deficiencies": "sum"
    })
    # add in state names
    .merge(
        state_lookup,
        on = "provider_state"
    )
    # calc
    .assign(
        percent_infection = lambda frame: frame["infection deficiencies"]/frame["total_facilities"] * 100 ,
        percent_sores = lambda frame: frame["sore deficiencies"]/frame["total_facilities"] * 100,
        percent_med = lambda frame: frame["med deficiencies"]/frame["total_facilities"] * 100
    )
    .sort_values(
        "percent_infection",
        ascending = False
    )
    .head(15)
)

,provider_state,infection deficiencies,sore deficiencies,med deficiencies,total_facilities,percent_infection,percent_sores,percent_med
11,GU,1.0,0.0,0.0,1,100.000000,0.000000,0.000000
4,CA,964.0,317.0,137.0,1193,80.804694,26.571668,11.483655
51,WV,99.0,25.0,5.0,123,80.487805,20.325203,4.065041
14,ID,66.0,34.0,6.0,82,80.487805,41.463415,7.317073
33,NM,55.0,21.0,10.0,71,77.464789,29.577465,14.084507
12,HI,34.0,9.0,4.0,44,77.272727,20.454545,9.090909
23,MI,341.0,185.0,51.0,442,77.149321,41.855204,11.538462
47,VA,216.0,109.0,77.0,286,75.524476,38.111888,26.923077
25,MO,390.0,123.0,62.0,522,74.712644,23.563218,11.877395
34,NV,50.0,20.0,10.0,67,74.626866,29.850746,14.925373


## Examining repeat offenders

In [26]:
# get a bool for each deficiency-cycle
per_cycle = (
    df
    .loc[
        lambda x: x["standard_deficiency"] == "Y"
    ]
    .groupby(['federal_provider_number', 'inspection_cycle']).pipe( lambda grp: pd.DataFrame({
        "infection": grp["has_infection"].sum(),
        "ulcer": grp["has_ulcer"].sum(),
        "med": grp["has_med"].sum()
    })
    .unstack()
))

per_cycle.head(5)

infection                ulcer                  med  \
inspection_cycle                1      2      3      1      2      3      1   
federal_provider_number                                                       
015009                       True   True    NaN  False  False    NaN  False   
015010                       True   True  False  False  False  False  False   
015012                       True   True   True  False  False  False  False   
015014                       True  False   True  False  False  False  False   
015015                      False  False  False  False  False  False  False   

                                       
inspection_cycle             2      3  
federal_provider_number                
015009                   False    NaN  
015010                   False  False  
015012                   False  False  
015014                   False  False  
015015                   False  False

In [27]:
# Helper function for finding repeat offenders
def repeats(deficiency, cycles):
    total = surveys.loc[
        lambda x: x["inspection_cycle"].isin
        ( [each for each in range(1, cycles + 1)])
    ]["federal_provider_number"].nunique()
    
    if cycles == 2:
        return (
            per_cycle
            [deficiency]
            .loc[
                lambda x: (x["1"] == True) & (x["2"] == True)
            ]
            .pipe(len)
        ) / total * 100
    if cycles == 3:
        return (
            per_cycle
            [deficiency]
            .loc[
                lambda x: (x["1"] == True) & (x["2"] == True) & (x["3"] == True)
            ]
            .pipe(len)
        ) / total * 100
        

Percent repeat infection control deficiencies?

In [28]:
repeats("infection", 2)

19.545513401527902

In [29]:
repeats("infection", 3)

9.78894212093746

Percent repeat pressure sore deficiencies?

In [30]:
repeats("ulcer", 2)

2.861582286676162

In [31]:
repeats("ulcer", 3)

0.9711252104104623

Percent repeat medication errors?

In [32]:
repeats("med", 2)

0.5049851094134403

In [33]:
repeats("med", 3)

0.12300919331865855

---

---

---